<a href="https://colab.research.google.com/github/urmzd/lepus-classifier/blob/main/notebooks/example_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/rabbit-classifier/
!git clone https://github.com/urmzd/rabbit-classifier
!pip install loguru pipetools pytorch-lightning wandb

Cloning into 'rabbit-classifier'...
remote: Enumerating objects: 873, done.
remote: Counting objects: 100% (873/873), done.
remote: Compressing objects: 100% (533/533), done.
remote: Total 873 (delta 450), reused 595 (delta 242), pack-reused 0
Receiving objects: 100% (873/873), 51.64 MiB | 8.10 MiB/s, done.
Resolving deltas: 100% (450/450), done.


In [2]:
import sys
SRC_PATH = "/content/rabbit-classifier"
sys.path.insert(0, SRC_PATH)

In [5]:
from src.bootstrap import bootstrap, TrainerFactory
from pathlib import Path
from pytorch_lightning.loggers import WandbLogger
import wandb
import pytorch_lightning as pl
import torch
from torchmetrics import Accuracy
import torch.nn.functional as F

class SampleModel(pl.LightningModule):
    def __init__(self, n_targets=2) -> None:
        super().__init__()
        # O: H/2, W/2
        self.layer_1 = torch.nn.Conv2d(1, 15, 2, 2)
        # O: H/2, W/2
        self.layer_2 = torch.nn.MaxPool2d(2, 2)
        self.layer_3 = torch.nn.ReLU()
        self.layer_4 = torch.nn.Flatten(1, -1)
        self.layer_5 = torch.nn.Linear(15 * 50 * 50, 500)
        self.layer_6 = torch.nn.Linear(500, n_targets)
        self.layer_7 = torch.nn.LogSoftmax()
        self.test_acc = Accuracy()
        self.train_acc = Accuracy()
        self.save_hyperparameters()

    def forward(self, x):
        x_1 = self.layer_1(x)
        x_2 = self.layer_2(x_1)
        x_3 = self.layer_3(x_2)
        x_4 = self.layer_4(x_3)
        x_5 = self.layer_5(x_4)
        x_6 = self.layer_6(x_5)
        x_7 = self.layer_7(x_6)
        result = x_7
        return result

    def training_step(self, batch, batch_idx) -> torch.Tensor:
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y)
        self.train_acc(y_hat, y)
        self.log("train_acc", self.train_acc)
        self.log("train_loss", loss)
        return loss

    def test_step(self, batch, batch_idx) -> None:
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y)
        self.test_acc(y_hat, y)
        self.log("test_acc", self.test_acc)
        self.log("test_loss", loss)

    def validation_step(self, batch, batch_idx) -> torch.Tensor:
        x, y = batch
        y_hat = self.forward(x)
        val_loss = F.cross_entropy(y_hat, y)
        return val_loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)


trainer_factory = TrainerFactory(strategy=None, max_epochs=50, logger=WandbLogger(project="rabbit-classifier"))

bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/rabbit-classifier/resources/data.csv"),
          model=SampleModel())

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type       | Params
-----------------------------------------
0 | layer_1   | Conv2d     | 75    
1 | layer_2   | MaxPool2d  | 0     
2 | layer_3   | ReLU       | 0     
3 | layer_4   | Flatten    | 0     
4 | layer_5   | Linear     | 18.8 M
5 | layer_6   | Linear     | 1.0 K 
6 | layer_7   | LogSoftmax | 0     
7 | test_acc  | Accuracy   | 0     
8 | train_acc | Accuracy   | 0     
-----------------------------------------
18.8 M    Trainable params
0         Non-trainable params
18.8 M    Total params
75.006    Total estimated model params size (MB)
2022-03-31 01:42:55.064 | INFO     | src.data.data_handler:on_advance_start:195 - STARTING FOLD 0
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connec

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    0.5
        test_loss           323.57659912109375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      mean_test_acc                 0.5
     mean_test_loss         1314.2117919921875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────



epoch,▄██▄██▄██▄██▄██▁
mean_test_acc,▁
mean_test_loss,▁
test_acc,▁▁▁█▁
test_loss,█▁▂▁▁
train_acc,██▁▁██████
train_loss,▁▁██▁▁▁▁▂▁
trainer/global_step,▁▂▂▃▃▃▄▅▅▆▆▆▇███
epoch,0
mean_test_acc,0.5
mean_test_loss,1314.21179
